# ABU量化系统使用文档 

<center>
        <img src="./image/abu_logo.png" alt="" style="vertical-align:middle;padding:10px 20px;"><font size="6" color="black"><b>第16节 UMP主裁交易决策</b></font>
</center>

-----------------


作者: 阿布

阿布量化版权所有 未经允许 禁止转载

[abu量化系统github地址](https://github.com/bbfamily/abu) (欢迎+star)

[本节ipython notebook](https://github.com/bbfamily/abu/tree/master/abupy_lecture)

上一节示例了ump角度主裁的训练分解步骤，对任何比赛一个裁判是远远不够的，本节将训练更多的裁判, 首先导入abupy中本节使用的模块：

In [1]:
# 基础库导入

from __future__ import print_function
from __future__ import division

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
import sys
# 使用insert 0即只使用github，避免交叉使用了pip安装的abupy，导致的版本不一致问题
sys.path.insert(0, os.path.abspath('../'))
import abupy

# 使用沙盒数据，目的是和书中一样的数据环境
abupy.env.enable_example_env_ipython()

enable example env will only read RomDataBu/df_kl.h5


In [2]:
from abupy import AbuFactorAtrNStop, AbuFactorPreAtrNStop, AbuFactorCloseAtrNStop, AbuFactorBuyBreak, ABuProgress
from abupy import abu, EMarketTargetType, AbuMetricsBase, ABuMarketDrawing, AbuFuturesCn, ABuSymbolPd, AbuOrderPdProxy
from abupy import AbuUmpMainDeg, AbuUmpMainJump, AbuUmpMainPrice, AbuUmpMainWave, AbuFuturesCn, EStoreAbu, AbuML
from abupy import AbuUmpEdgeDeg, AbuUmpEdgePrice, AbuUmpEdgeWave, AbuUmpEdgeMul

受限于沙盒中数据限制，本节示例的相关性分析只限制在abupy内置沙盒数据中，完整示例以及代码请阅读《量化交易之路》中相关章节。

和上一节一样首先将内置沙盒中美股，A股，港股, 比特币，莱特币，期货市场中的symbol都列出来，然后组成训练集和测试集，买入卖出因子等相同设置:

In [3]:
us_choice_symbols = ['usTSLA', 'usNOAH', 'usSFUN', 'usBIDU', 'usAAPL', 'usGOOG', 'usWUBA', 'usVIPS']
cn_choice_symbols = ['002230', '300104', '300059', '601766', '600085', '600036', '600809', '000002', '002594']
hk_choice_symbols = ['hk03333', 'hk00700', 'hk02333', 'hk01359', 'hk00656', 'hk03888', 'hk02318']
tc_choice_symbols = ['btc', 'ltc']
# 期货市场的直接从AbuFuturesCn().symbo中读取
ft_choice_symbols = AbuFuturesCn().symbol.tolist()

# 训练集：沙盒中所有美股 ＋ 沙盒中所有A股 ＋ 沙盒中所有港股 ＋ 比特币
train_choice_symbols = us_choice_symbols + cn_choice_symbols +  hk_choice_symbols + tc_choice_symbols[:1]
# 测试集：沙盒中所有期货 ＋ 莱特币
test_choice_symbols = ft_choice_symbols + tc_choice_symbols[1:]

# 设置初始资金数
read_cash = 1000000
# 买入因子依然延用向上突破因子
buy_factors = [{'xd': 60, 'class': AbuFactorBuyBreak},
               {'xd': 42, 'class': AbuFactorBuyBreak}]

# 卖出因子继续使用上一节使用的因子
sell_factors = [
    {'stop_loss_n': 1.0, 'stop_win_n': 3.0,
     'class': AbuFactorAtrNStop},
    {'class': AbuFactorPreAtrNStop, 'pre_atr_n': 1.5},
    {'class': AbuFactorCloseAtrNStop, 'close_atr_n': 1.5}
]
# 回测生成买入时刻特征
abupy.env.g_enable_ml_feature = True

在运行完成第15节中相关内容后，使用load_abu_result_tuple读取上一节保存在本地的训练集数据：

In [4]:
abu_result_tuple_train = abu.load_abu_result_tuple(n_folds=2, store_type=EStoreAbu.E_STORE_CUSTOM_NAME, 
                                             custom_name='lecture_train')
orders_pd_train = abu_result_tuple_train.orders_pd
AbuMetricsBase.show_general(*abu_result_tuple_train, returns_cmp=True, only_info=True)

买入后卖出的交易数量:196
买入后尚未卖出的交易数量:9
胜率:59.6939%
平均获利期望:18.6899%
平均亏损期望:-7.1235%
盈亏比:4.4972
所有交易收益比例和:16.2396 
所有交易总盈亏和:2717948.4900 


please wait! load_pickle....: /Users/Bailey/abu/data/cache/n2_lecture_train_capital
please wait! load_pickle....: /Users/Bailey/abu/data/cache/n2_lecture_train_benchmark


### 1. 跳空主裁

上一节训练角度主裁的过程比较繁琐，实际上只是为了分步演示实现步骤，本节将使用类方法ump_main_clf_dump直接完成gmm分类，分类簇最优参数，分类簇筛选，本地保存分类器等操作, 由于本节示例使用的沙盒数据交易量太少，所以效果不会很好，本节示例的目的只是以最少量的数据，便于快速运行，快速实现，理解ump主裁的使用，20节之后示例使用ump训练全市场的交易，可以看到ump在回测中将会大大提升回测效果。

即实现上一节训练角度主裁的所有的操作实际上只需要运行下面这一行代码即可：

In [5]:
_ = AbuUmpMainDeg.ump_main_clf_dump(orders_pd_train, p_ncs=slice(20, 40, 1))

pid:56312 gmm fit:100.0%
pid:56312 done!
please wait! dump_pickle....: /Users/Bailey/abu/data/ump/ump_main_us_deg_main


下面我们通过ump_main_clf_dump()函数对AbuUmpMainJump完成跳空裁判的训练保存等一系列工作：

In [6]:
ump_jump = AbuUmpMainJump.ump_main_clf_dump(orders_pd_train, p_ncs=slice(20, 40, 1))
ump_jump.fiter.df.head()

pid:56312 gmm fit:100.0%
pid:56312 done!
please wait! dump_pickle....: /Users/Bailey/abu/data/ump/ump_main_us_jump_main


,result,buy_jump_up_power,buy_diff_down_days,buy_jump_down_power,buy_diff_up_days
2014-09-25,1,0.000,77,-1.369,0
2014-10-09,0,0.000,0,0.000,0
2014-10-17,1,1.778,0,0.000,41
2014-10-24,1,1.038,136,-13.570,2
2014-10-24,1,1.038,136,-13.570,2


上面通过ump_main_clf_dump()已经完成训练，筛选，本地保存分类器等工作，我们下面还是针对AbuUmpMainJump寻找宏观上合理的分类簇拦截交易解释。
AbuUmpMainJump中训练特征为：

* diff_up_days：距离这次交易最近一次向上跳空买入日期时间间隔
* diff_down_days：距离这次交易最近一次向下跳空买入日期时间间隔
* ump_up_power： 距离这次交易最近一次向上跳空能量
* jump_down_power： 距离这次交易最近一次向下跳空能量


### 2. 价格主裁

依然使用ump_main_clf_dump()函数完成价格主裁的训练，最优，保存等工作：

In [7]:
ump_price = AbuUmpMainPrice.ump_main_clf_dump(orders_pd_train, p_ncs=slice(20, 40, 1))
ump_price.fiter.df.head()

pid:56312 gmm fit:100.0%
pid:56312 done!
please wait! dump_pickle....: /Users/Bailey/abu/data/ump/ump_main_us_price_main


,result,buy_price_rank120,buy_price_rank90,buy_price_rank60,buy_price_rank252
2014-09-25,1,1.000,1.000,1.000,0.857
2014-10-09,0,0.992,0.989,0.983,0.798
2014-10-17,1,1.000,1.000,1.000,1.000
2014-10-24,1,1.000,1.000,1.000,1.000
2014-10-24,1,1.000,1.000,1.000,1.000


如上显示的特征为买入当天价格相对特征周期内所有价格排序的位置值，即比如特征周期为60天，那么如果买入当天价格为60天内最高，那么price_rank60=1.0；如果买入当天价格为60天内第30高价格，那么price_rank60=0.5。

可以看到值普遍比较大，这是因为使用42日、60日突破作为买入信号，这表示在短周期内大概率为rank最大值，长周期内也是大概率比较大的值

### 3.  波动主裁

依然使用ump_main_clf_dump()函数完成波动主裁的训练，最优，保存等工作：

In [8]:
ump_wave = AbuUmpMainWave.ump_main_clf_dump(orders_pd_train, p_ncs=slice(20, 40, 1))
ump_wave.fiter.df.head()

pid:56312 gmm fit:100.0%
pid:56312 done!
please wait! dump_pickle....: /Users/Bailey/abu/data/ump/ump_main_us_wave_main


,result,buy_wave_score1,buy_wave_score2,buy_wave_score3
2014-09-25,1,0.496,0.454,0.441
2014-10-09,0,0.230,0.072,-0.001
2014-10-17,1,0.743,0.374,0.232
2014-10-24,1,1.251,1.290,1.289
2014-10-24,1,1.251,1.290,1.289


显示的特征为三个周期内的价格波动率特征。

回测中生成价格波动率数值特征wave_score的代码在ABuMLFeature中，详情请自行查阅源代码，也可阅读《量化交易之路》`5.3.1 绘制股票的收益，及收益波动`章节查看代码片段


### 4. 验证主裁是否称职, 在abu系统中开启主裁拦截模式

首先使用测试集在不使用主裁拦截情况下进行回测，如下所示：

In [9]:
# 不使用主裁拦截
abupy.env.g_enable_ump_main_deg_block = False
abupy.env.g_enable_ump_main_jump_block = False
abupy.env.g_enable_ump_main_price_block = False
abupy.env.g_enable_ump_main_wave_block = False

abu_result_tuple_test, _ = abu.run_loop_back(read_cash,
                                                   buy_factors,
                                                   sell_factors,
                                                   start='2014-07-26',
                                                   end='2016-07-26',
                                                   choice_symbols=test_choice_symbols)
ABuProgress.clear_output()

In [10]:
AbuMetricsBase.show_general(*abu_result_tuple_test, returns_cmp=True, only_info=True)

买入后卖出的交易数量:247
买入后尚未卖出的交易数量:23
胜率:41.2955%
平均获利期望:9.7208%
平均亏损期望:-4.8754%
盈亏比:1.3725
所有交易收益比例和:2.8459 
所有交易总盈亏和:428259.7700 


同上相同参数进行测试集回测，相同的资金及策略参数等，唯一不同点在于开启主裁拦截，代码如下所示：

In [11]:
abupy.env.g_enable_ump_main_deg_block = True
abupy.env.g_enable_ump_main_jump_block = True
abupy.env.g_enable_ump_main_price_block = True
abupy.env.g_enable_ump_main_wave_block = True

abu_result_tuple_test_ump, _ = abu.run_loop_back(read_cash,
                                                   buy_factors,
                                                   sell_factors,
                                                   start='2014-07-26',
                                                   end='2016-07-26',
                                                   choice_symbols=test_choice_symbols)
ABuProgress.clear_output()

In [12]:
AbuMetricsBase.show_general(*abu_result_tuple_test_ump, returns_cmp=True, only_info=True)

买入后卖出的交易数量:154
买入后尚未卖出的交易数量:18
胜率:44.8052%
平均获利期望:10.9532%
平均亏损期望:-4.1308%
盈亏比:2.1145
所有交易收益比例和:4.0465 
所有交易总盈亏和:617759.7700 


使用AbuOrderPdProxy可以查看被拦截的交易:

In [13]:
proxy = AbuOrderPdProxy(abu_result_tuple_test.orders_pd)
with proxy.proxy_work(abu_result_tuple_test_ump.orders_pd) as (order1, order2):
     block_order = order1 - order2
print('正确拦截失败的交易数量{}, 错误拦截的交易数量{}'.format(block_order.result.value_counts()[-1], block_order.result.value_counts()[1]))
block_order.head()

正确拦截失败的交易数量60, 错误拦截的交易数量33


,buy_date,buy_price,buy_cnt,buy_factor,symbol,buy_pos,buy_type_str,expect_direction,sell_type_extra,sell_date,...,sell_wave_score2,sell_wave_score3,sell_diff_up_days,sell_jump_down_power,sell_jump_up_power,sell_diff_down_days,profit_cg,profit_cg_hunder,keep_days,difference
2014-10-15,20141015,4744.500,40.0,AbuFactorBuyBreak:42,JD0,AbuAtrPosition,call,1.0,AbuFactorPreAtrNStop:pre_atr=1.5,20141104,...,0.521,0.580,25.0,-2.135,2.633,290.0,-0.0317,-3.1721,20,True
2014-10-29,20141029,2393.500,80.0,AbuFactorBuyBreak:42,C0,AbuAtrPosition,call,1.0,AbuFactorCloseAtrNStop:close_atr_n=1.5,20141215,...,0.499,0.627,59.0,-3.602,1.262,64.0,0.0025,0.2507,47,True
2014-12-31,20141231,129.775,1500.0,AbuFactorBuyBreak:60,BB0,AbuAtrPosition,call,1.0,AbuFactorAtrNStop:stop_loss=1.0,20150115,...,1.030,0.850,0.0,-1.557,0.000,14.0,-0.1179,-11.7896,15,True
2014-12-31,20141231,129.775,1500.0,AbuFactorBuyBreak:42,BB0,AbuAtrPosition,call,1.0,AbuFactorAtrNStop:stop_loss=1.0,20150115,...,1.030,0.850,0.0,-1.557,0.000,14.0,-0.1179,-11.7896,15,True
2015-01-21,20150121,3853.000,45.0,AbuFactorBuyBreak:42,AG0,AbuAtrPosition,call,1.0,AbuFactorAtrNStop:stop_loss=1.0,20150202,...,1.397,1.417,106.0,-1.304,1.481,88.0,-0.0628,-6.2808,12,True


由于本节的示例只是从沙盒数据中的为数不多的交易进行训练，且测试集交易数量也不多，所以效果确实一般（在20节后的文档会陆续讲解美股，A股，港股等全市场回测后进行ump训练，回测测试集的示例）可以从对比中发现，大约拦截了一百多笔交易，正确拦截的数量比错误拦截的要多，胜率和盈亏并不理想，拦截了大量的交易可以节省佣金，降低交易数量是最好的优化。

想要更多的提升可以尝试如下方面：

1. 编写更多的裁判，寻找更多的特征
2. 在make_ump_block_decision()中组织裁判进行更复杂的综合裁决，进行裁判之间的拦截配合
3. 为每个裁判通过统计赋予裁决权重，进行综合裁决
4. 训练更多交易数据，更多策略来提升主裁的拦截水平及拦截认知范围

第四点是最有效果且最正确的解决途径，因为依赖统计机器学习的算法当数据越充足时效果越好。

第一点会在之后的章节示例编写自定义ump裁判中讲解，下面首先示例第二点。

### 5. 组织裁判进行更复杂的综合裁决

如下继承买入策略AbuFactorBuyBreak，覆写make_ump_block_decision方法，如下所示：

In [14]:
class AbuFactorBuyBreakUmpDemo(AbuFactorBuyBreak):
    """扩展AbuFactorBuyBreak组织裁判进行更复杂的综合裁决"""
    
    def make_ump_block_decision(self, ml_feature_dict):
        ump = self.ump_manger
        # 统计角度主裁对应这次交易命中的分类簇个数
        deg_hit_cnt = ump.ump_main_deg.predict_hit_kwargs(**ml_feature_dict)
        # 统计跳空主裁对应这次交易命中的分类簇个数
        jump_hit_cnt = ump.ump_main_jump.predict_hit_kwargs(**ml_feature_dict)
        # 统计波动主裁对应这次交易命中的分类簇个数
        wave_hit_cnt = ump.ump_main_wave.predict_hit_kwargs(**ml_feature_dict)
        # 统计价格主裁对应这次交易命中的分类簇个数
        price_hit_cnt = ump.ump_main_price.predict_hit_kwargs(**ml_feature_dict)
        # 进行裁判之间的拦截配合, 简单示例，只要加起来大于2个就算配合成功，拦截
        if deg_hit_cnt + jump_hit_cnt + wave_hit_cnt + price_hit_cnt > 2:
            return True
        return False

# 通过import的方式导入AbuFactorBuyBreakUmpDemo
# 因为在windows系统上，启动并行后，在ipython notebook中定义的类会在子进程中无法找到
from abupy import AbuFactorBuyBreakUmpDemo

上面编写的代码实现了在策略的make_ump_block_decision中组织裁判进行更复杂的综合裁决，进行裁判之间的拦截配合，实际上这里的make_ump_block_decision编写的太过简单，只是为了简单示例，具体实现请参考AbuUmpManager中ump_block函数，下面使用AbuFactorBuyBreakUmpDemo做为买入因子，参数还是60，42进行回测如下所示：

In [15]:
buy_factors = [{'xd': 60, 'class': AbuFactorBuyBreakUmpDemo},
               {'xd': 42, 'class': AbuFactorBuyBreakUmpDemo}]

abu_result_tuple_test_ump_mul, _ = abu.run_loop_back(read_cash,
                                                   buy_factors,
                                                   sell_factors,
                                                   start='2014-07-26',
                                                   end='2016-07-26',
                                                   choice_symbols=test_choice_symbols)
ABuProgress.clear_output()

In [16]:
AbuMetricsBase.show_general(*abu_result_tuple_test_ump_mul, returns_cmp=True, only_info=True)

买入后卖出的交易数量:167
买入后尚未卖出的交易数量:18
胜率:43.1138%
平均获利期望:11.7044%
平均亏损期望:-4.4000%
盈亏比:2.0418
所有交易收益比例和:4.2471 
所有交易总盈亏和:673707.2700 


下面示例上述说的第三点：为每个裁判通过统计赋予裁决权重，进行综合裁决。

### 6. 让裁判自己学习怎么配合，自己做出最正确的判断

即是可以再次根据裁判之间的配合数据进行训练学习，让裁判自己学习怎么配合，自己做出最正确的判断，而不是像上面的示例使用固定值2来做为裁决阀值，如下编写AbuFactorBuyBreakReocrdHitDemo类似AbuFactorBuyBreakUmpDemo但是不对交易进行决策，只是把每一个裁判的对应交易命中的分类簇个数进行记录，更新在特征数据里，如下所示：

In [17]:
class AbuFactorBuyBreakReocrdHitDemo(AbuFactorBuyBreak):
    def make_ump_block_decision(self, ml_feature_dict):
        ump = self.ump_manger
        # 统计角度主裁对应这次交易命中的分类簇个数
        deg_hit_cnt = ump.ump_main_deg.predict_hit_kwargs(**ml_feature_dict)
        # 统计跳空主裁对应这次交易命中的分类簇个数
        jump_hit_cnt = ump.ump_main_jump.predict_hit_kwargs(**ml_feature_dict)
        # 统计波动主裁对应这次交易命中的分类簇个数
        wave_hit_cnt = ump.ump_main_wave.predict_hit_kwargs(**ml_feature_dict)
        # 统计价格主裁对应这次交易命中的分类簇个数
        price_hit_cnt = ump.ump_main_price.predict_hit_kwargs(**ml_feature_dict)
        
        ml_feature_dict.update({'deg_hit_cnt': deg_hit_cnt, 'jump_hit_cnt': jump_hit_cnt, 
                                'wave_hit_cnt': wave_hit_cnt, 'price_hit_cnt': price_hit_cnt})
        
        return False
    
# 通过import的方式导入AbuFactorBuyBreakReocrdHitDemo，
# 因为在windows系统上，启动并行后，在ipython notebook中定义的类会在子进程中无法找到
from abupy import AbuFactorBuyBreakReocrdHitDemo

如下使用AbuFactorBuyBreakReocrdHitDemo进行回测，如下所示：

In [18]:
buy_factors = [{'xd': 60, 'class': AbuFactorBuyBreakReocrdHitDemo},
               {'xd': 42, 'class': AbuFactorBuyBreakReocrdHitDemo}]

abu_result_tuple_test_ump_record, _ = abu.run_loop_back(read_cash,
                                                   buy_factors,
                                                   sell_factors,
                                                   start='2014-07-26',
                                                   end='2016-07-26',
                                                   choice_symbols=test_choice_symbols)
ABuProgress.clear_output()

下面把刚才AbuFactorBuyBreakReocrdHitDemo在回测中记录的数据和交易的最终结果result进行组合，形成hit_df，如下所示：

In [19]:
hit_df = pd.DataFrame()
def make_hit_df(order):
    global hit_df
    if order.result != 0:
        hit = pd.DataFrame([order.result, order.ml_features['deg_hit_cnt'], 
            order.ml_features['jump_hit_cnt'], 
            order.ml_features['wave_hit_cnt'],
            order.ml_features['price_hit_cnt']], 
            index=['result', 'deg_hit_cnt', 'jump_hit_cnt', 'wave_hit_cnt', 'price_hit_cnt']).T
        hit_df = hit_df.append(hit)
        
_ = abu_result_tuple_test_ump_record.orders_pd.apply(make_hit_df, axis=1)
hit_df.head()

,result,deg_hit_cnt,jump_hit_cnt,wave_hit_cnt,price_hit_cnt
0,-1,0,0,0,0
0,-1,0,0,0,0
0,-1,0,0,0,0
0,-1,0,7,0,0
0,-1,0,0,0,0


上面形成的hit_df的第一列result可做为有监督学习的y，其它的列可做为x特征列，下面使用AbuML直接封装数据：

备注：abupy中的机器学习模块请阅读：第十二节 机器学习与比特币示例

In [20]:
hd_np = hit_df.as_matrix()
y = hd_np[:, 0]
x = hd_np[:, 1:]
hit_ml = AbuML(x, y, hit_df)
hit_ml.fit()

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

下面继续继承AbuFactorBuyBreak复写make_ump_block_decision，区别是使用hit_ml对几个裁判这次交易命中的分类簇个数组成矢量特征进行predict，拦截预测结果为-1的交易，代码如下所示：

In [21]:
class AbuFactorBuyBreakHitPredictDemo(AbuFactorBuyBreak):
    """扩展AbuFactorBuyBreak组织裁判进行更复杂的综合裁决"""

    def _init_self(self, **kwargs):
        """
            与AbuFactorBuyBreak基本相同，唯一区别是关键子参数中添加了通过AbuFactorBuyBreakUmpDemo记录训练好的决策器
            self.hit_ml = kwargs['hit_ml']
        """
        super(AbuFactorBuyBreakHitPredictDemo, self)._init_self(**kwargs)
        # 添加了通过AbuFactorBuyBreakUmpDemo记录训练好的决策器
        self.hit_ml = kwargs['hit_ml']

    def make_ump_block_decision(self, ml_feature_dict):
        ump = self.ump_manger
        # 统计角度主裁对应这次交易命中的分类簇个数
        deg_hit_cnt = ump.ump_main_deg.predict_hit_kwargs(**ml_feature_dict)
        # 统计跳空主裁对应这次交易命中的分类簇个数
        jump_hit_cnt = ump.ump_main_jump.predict_hit_kwargs(**ml_feature_dict)
        # 统计波动主裁对应这次交易命中的分类簇个数
        wave_hit_cnt = ump.ump_main_wave.predict_hit_kwargs(**ml_feature_dict)
        # 统计价格主裁对应这次交易命中的分类簇个数
        price_hit_cnt = ump.ump_main_price.predict_hit_kwargs(**ml_feature_dict)

        result = self.hit_ml.predict([deg_hit_cnt, jump_hit_cnt, wave_hit_cnt, price_hit_cnt])[0]
        if result == -1:
            return True
        return False
    
# 通过import的方式导入AbuFactorBuyBreakHitPredictDemo
# 因为在windows系统上，启动并行后，在ipython notebook中定义的类会在子进程中无法找到
from abupy import AbuFactorBuyBreakHitPredictDemo

使用AbuFactorBuyBreakHitPredictDemo进行回测，注意在构造策略字典序列的时候使用了刚刚训练好的hit_ml，如下所示：

In [22]:
buy_factors = [{'hit_ml':hit_ml, 'xd': 60, 'class': AbuFactorBuyBreakHitPredictDemo},
               {'hit_ml':hit_ml, 'xd': 42, 'class': AbuFactorBuyBreakHitPredictDemo}]

abu_result_tuple_test_ump_predict, _ = abu.run_loop_back(read_cash,
                                                   buy_factors,
                                                   sell_factors,
                                                   start='2014-07-26',
                                                   end='2016-07-26',
                                                   choice_symbols=test_choice_symbols)
ABuProgress.clear_output()

以下度量结果胜率，盈亏比都达到很高，因为用回测的数据进行训练后再次反过来指导回测，结果是没有意义的，这里的示例只是为了容易理解什么叫做：**让裁判自己学习怎么配合，自己做出最正确的判断**，更详细完整的示例会在之后的章节中示例讲解，请关注公众号的更新提醒。

In [23]:
AbuMetricsBase.show_general(*abu_result_tuple_test_ump_predict, returns_cmp=True, only_info=True)

买入后卖出的交易数量:8
买入后尚未卖出的交易数量:3
胜率:100.0000%
平均获利期望:16.4870%
平均亏损期望:0.0000%
盈亏比:0.0000
所有交易收益比例和:1.3190 
所有交易总盈亏和:219657.5000 


小结：本节的示例与《量化交易之路》中讲解的主裁部分内容互为补充，请对照阅读。

#### abu量化文档目录章节

1. [择时策略的开发](http://www.abuquant.com/lecture/lecture_1.html)
2. [择时策略的优化](http://www.abuquant.com/lecture/lecture_2.html)
3. [滑点策略与交易手续费](http://www.abuquant.com/lecture/lecture_3.html)
4. [多支股票择时回测与仓位管理](http://www.abuquant.com/lecture/lecture_4.html)
5. [选股策略的开发](http://www.abuquant.com/lecture/lecture_5.html)
6. [回测结果的度量](http://www.abuquant.com/lecture/lecture_6.html)
7. [寻找策略最优参数和评分](http://www.abuquant.com/lecture/lecture_7.html)
8. [A股市场的回测](http://www.abuquant.com/lecture/lecture_8.html)
9. [港股市场的回测](http://www.abuquant.com/lecture/lecture_9.html)
10. [比特币，莱特币的回测](http://www.abuquant.com/lecture/lecture_10.html)
11. [期货市场的回测](http://www.abuquant.com/lecture/lecture_11.html)
12. [机器学习与比特币示例](http://www.abuquant.com/lecture/lecture_12.html)
13. [量化技术分析应用](http://www.abuquant.com/lecture/lecture_13.html)
14. [量化相关性分析应用](http://www.abuquant.com/lecture/lecture_14.html)
15. [量化交易和搜索引擎](http://www.abuquant.com/lecture/lecture_15.html)
16. [UMP主裁交易决策](http://www.abuquant.com/lecture/lecture_16.html)
17. [UMP边裁交易决策](http://www.abuquant.com/lecture/lecture_17.html)
18. [自定义裁判决策交易](http://www.abuquant.com/lecture/lecture_18.html)
19. [数据源](http://www.abuquant.com/lecture/lecture_19.html)
20. [A股全市场回测](http://www.abuquant.com/lecture/lecture_20.html)
21. [A股UMP决策](http://www.abuquant.com/lecture/lecture_21.html)
22. [美股全市场回测](http://www.abuquant.com/lecture/lecture_22.html)
23. [美股UMP决策](http://www.abuquant.com/lecture/lecture_23.html)

abu量化系统文档教程持续更新中，请关注公众号中的更新提醒。

#### 《量化交易之路》目录章节及随书代码地址

1. [第二章 量化语言——Python](https://github.com/bbfamily/abu/tree/master/ipython/第二章-量化语言——Python.ipynb)
2. [第三章 量化工具——NumPy](https://github.com/bbfamily/abu/tree/master/ipython/第三章-量化工具——NumPy.ipynb)
3. [第四章 量化工具——pandas](https://github.com/bbfamily/abu/tree/master/ipython/第四章-量化工具——pandas.ipynb)
4. [第五章 量化工具——可视化](https://github.com/bbfamily/abu/tree/master/ipython/第五章-量化工具——可视化.ipynb)
5. [第六章 量化工具——数学：你一生的追求到底能带来多少幸福](https://github.com/bbfamily/abu/tree/master/ipython/第六章-量化工具——数学.ipynb)
6. [第七章 量化系统——入门：三只小猪股票投资的故事](https://github.com/bbfamily/abu/tree/master/ipython/第七章-量化系统——入门.ipynb)
7. [第八章 量化系统——开发](https://github.com/bbfamily/abu/tree/master/ipython/第八章-量化系统——开发.ipynb)
8. [第九章 量化系统——度量与优化](https://github.com/bbfamily/abu/tree/master/ipython/第九章-量化系统——度量与优化.ipynb)
9. [第十章 量化系统——机器学习•猪老三](https://github.com/bbfamily/abu/tree/master/ipython/第十章-量化系统——机器学习•猪老三.ipynb)
10. [第十一章 量化系统——机器学习•ABU](https://github.com/bbfamily/abu/tree/master/ipython/第十一章-量化系统——机器学习•ABU.ipynb)
11. [附录A 量化环境部署](https://github.com/bbfamily/abu/tree/master/ipython/附录A-量化环境部署.ipynb)
12. [附录B 量化相关性分析](https://github.com/bbfamily/abu/tree/master/ipython/附录B-量化相关性分析.ipynb)
13. [附录C 量化统计分析及指标应用](https://github.com/bbfamily/abu/tree/master/ipython/附录C-量化统计分析及指标应用.ipynb)

[更多阿布量化量化技术文章](http://www.abuquant.com/article)


更多关于量化交易相关请阅读[《量化交易之路》](http://www.abuquant.com/books/quantify-trading-road.html)

更多关于量化交易与机器学习相关请阅读[《机器学习之路》](http://www.abuquant.com/books/machine-learning-road.html)

更多关于abu量化系统请关注微信公众号: abu_quant

![](./image/qrcode.jpg)